### Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd

/content


In [3]:
PATH_TO_FOLDER = '/content/drive/MyDrive/DL_project/EXPERIMENTS'

In [4]:
%cd {PATH_TO_FOLDER}

/content/drive/MyDrive/DL_project/EXPERIMENTS


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
from models import MLP, NAC, NALU

In [7]:
import math
import random
import numpy as np
from tqdm.notebook import tqdm
from copy import deepcopy

import torch
import torch.nn as nn
import torch.nn.functional as F

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
%%capture
!pip install wandb --upgrade

In [10]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### INTERPOLATION Parameters

In [11]:
NAMES = dict(
    project = 'INTERPOLATION',
    tags = 'preparation'
)

MODEL_PARAMETERS = dict(
    in_dim=2,
    out_dim=1,
    NUM_LAYERS = 2,
    HIDDEN_DIM = 2,
)

ARITHMETIC_PARAMETERS = dict(
    num_train=500, 
    num_test=50,
    dim=100, 
    num_sum=5,
    RANGE = [5, 10]
)

TRAIN_PARAMETERS = dict(
                        LEARNING_RATE = 1e-2,
                        # NUM_ITERS = int(1e5),
                        NUM_ITERS = int(50000),
                        activation='relu6'
                        )

ARITHMETIC_FUNCTIONS = {
    'add': lambda x, y: x + y,
    'sub': lambda x, y: x - y,
    'mul': lambda x, y: x * y,
    'div': lambda x, y: x / y,
    'squared': lambda x, y: torch.pow(x, 2),
    'root': lambda x, y: torch.sqrt(x)
    }

MODELS = {
        'MLP': MLP, 
        'NAC': NAC, 
        'NALU': NALU
        }

In [ ]:
# def get_id():
#     id_dct = {}
#     for i, key in enumerate(ARITHMETIC_FUNCTIONS.keys()):
#         id_dct[key] = i
#     return id_dct

# FUNC_id = get_id()

### Config for Wandb

In [12]:
def create_config(model, function):
    config = {'model': model,
              'function': function}
              
    models = MODEL_PARAMETERS.copy()
    funcs = ARITHMETIC_PARAMETERS.copy()
    params = TRAIN_PARAMETERS.copy()
    names = NAMES.copy()

    for key, value in models.items():
        config[key] = value
    for key, value in funcs.items():
        config[key] = value
    for key, value in params.items():
        config[key] = value
    for key, value in names.items():
        config[key] = value

    return config

### Example of config

In [13]:
config = create_config('MLP', 'add')

In [15]:
config

{'HIDDEN_DIM': 2,
 'LEARNING_RATE': 0.01,
 'NUM_ITERS': 50000,
 'NUM_LAYERS': 2,
 'RANGE': [5, 10],
 'activation': 'relu6',
 'dim': 100,
 'function': 'add',
 'in_dim': 2,
 'model': 'MLP',
 'num_sum': 5,
 'num_test': 50,
 'num_train': 500,
 'out_dim': 1,
 'project': 'INTERPOLATION',
 'tags': 'preparation'}

### Iteration on models and arithmetic functions

In [17]:
mdls = MODELS.keys()
fncts = ARITHMETIC_FUNCTIONS.keys()

for function in fncts:
    for arch in mdls:
        config = create_config(arch, function)
        # Build, train and analyze the model with the pipeline
        model = model_pipeline(config)

wandb: Currently logged in as: galmitr (use `wandb login --relogin` to force relogin)


	1/50000: loss: 3748.6340332 - mea: 61.1039314
	1001/50000: loss: 0.7828440 - mea: 0.8812203
	2001/50000: loss: 0.5584386 - mea: 0.7453416
	3001/50000: loss: 0.5582510 - mea: 0.7457025
	4001/50000: loss: 0.5568953 - mea: 0.7451655
	5001/50000: loss: 0.5557318 - mea: 0.7446712
	6001/50000: loss: 0.5547053 - mea: 0.7442012
	7001/50000: loss: 0.5537956 - mea: 0.7437565
	8001/50000: loss: 0.5529677 - mea: 0.7433252
	9001/50000: loss: 0.5522817 - mea: 0.7429571
	10001/50000: loss: 0.5516057 - mea: 0.7425703
	11001/50000: loss: 0.5510343 - mea: 0.7422344
	12001/50000: loss: 0.5506465 - mea: 0.7420072
	13001/50000: loss: 0.5501085 - mea: 0.7416670
	14001/50000: loss: 0.5497105 - mea: 0.7414125
	15001/50000: loss: 0.5493234 - mea: 0.7411587
	16001/50000: loss: 0.5489950 - mea: 0.7409395
	17001/50000: loss: 0.5485813 - mea: 0.7406591
	18001/50000: loss: 0.5482658 - mea: 0.7404423
	19001/50000: loss: 0.5478176 - mea: 0.7401340
	20001/50000: loss: 0.5476298 - mea: 0.7400004
	21001/50000: loss: 0.

epoch,49000
loss_(mse),0.54138
mean_absolute_error,0.73564
_runtime,30
_timestamp,1623439782
_step,50


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss_(mse),█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_absolute_error,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


	1/50000: loss: 7029.0747070 - mea: 83.6875458
	1001/50000: loss: 0.0033662 - mea: 0.0468918
	2001/50000: loss: 0.0192659 - mea: 0.1385377
	3001/50000: loss: 0.0172079 - mea: 0.1308930
	4001/50000: loss: 0.0145552 - mea: 0.1203790
	5001/50000: loss: 0.0122746 - mea: 0.1105449
	6001/50000: loss: 0.0103421 - mea: 0.1014696
	7001/50000: loss: 0.0087115 - mea: 0.0931267
	8001/50000: loss: 0.0073053 - mea: 0.0852791
	9001/50000: loss: 0.0061372 - mea: 0.0781640
	10001/50000: loss: 0.0051513 - mea: 0.0716105
	11001/50000: loss: 0.0043264 - mea: 0.0656273
	12001/50000: loss: 0.0036246 - mea: 0.0600686
	13001/50000: loss: 0.0030399 - mea: 0.0550108
	14001/50000: loss: 0.0025570 - mea: 0.0504529
	15001/50000: loss: 0.0021463 - mea: 0.0462238
	16001/50000: loss: 0.0017937 - mea: 0.0422569
	17001/50000: loss: 0.0015088 - mea: 0.0387558
	18001/50000: loss: 0.0012754 - mea: 0.0356329
	19001/50000: loss: 0.0010640 - mea: 0.0325461
	20001/50000: loss: 0.0008965 - mea: 0.0298743
	21001/50000: loss: 0.

...

epoch,49000
loss_(mse),0.03108
mean_absolute_error,0.13727
_runtime,32
_timestamp,1623440547
_step,50


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss_(mse),█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_absolute_error,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


	1/50000: loss: 8.3164883 - mea: 2.8119428
	1001/50000: loss: 0.0154093 - mea: 0.1205957
	2001/50000: loss: 0.0128875 - mea: 0.1127877
	3001/50000: loss: 0.0110573 - mea: 0.1047929
	4001/50000: loss: 0.0094072 - mea: 0.0966364
	5001/50000: loss: 0.0078510 - mea: 0.0882059
	6001/50000: loss: 0.0063535 - mea: 0.0791773
	7001/50000: loss: 0.0048988 - mea: 0.0689847
	8001/50000: loss: 0.0038040 - mea: 0.0603444
	9001/50000: loss: 0.0033137 - mea: 0.0558943
	10001/50000: loss: 0.0031482 - mea: 0.0544466
	11001/50000: loss: 0.0029410 - mea: 0.0526236
	12001/50000: loss: 0.0027793 - mea: 0.0511843
	13001/50000: loss: 0.0026313 - mea: 0.0498328
	14001/50000: loss: 0.0024966 - mea: 0.0485720
	15001/50000: loss: 0.0023721 - mea: 0.0473766
	16001/50000: loss: 0.0022549 - mea: 0.0462199
	17001/50000: loss: 0.0021429 - mea: 0.0450836
	18001/50000: loss: 0.0020391 - mea: 0.0440032
	19001/50000: loss: 0.0019404 - mea: 0.0429485
	20001/50000: loss: 0.0018477 - mea: 0.0419324
	21001/50000: loss: 0.0017

epoch,49000
loss_(mse),0.00083
mean_absolute_error,0.02852
_runtime,63
_timestamp,1623440617
_step,50


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss_(mse),█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_absolute_error,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


### Wandb pipeline

In [16]:
"""
pipeline for training model and tracking weights, gradients and metrics via Wandb
"""

def model_pipeline(hyperparameters):
    with wandb.init(project=f"{hyperparameters['project']}",
                    group=f"{hyperparameters['function']}", 
                    job_type=f"{hyperparameters['model']}",
                    name=f"{hyperparameters['function']}_{hyperparameters['model']}",
                    tags=f"{hyperparameters['tags']}",
                    config=hyperparameters):
        config = wandb.config

        random_model, model, data, criterion, optimizer = make(config)
        # print(model)

        # data
        train_data, test_data = data
 
        # random_model
        random_mea = []
        for i in range(100):
            abs = test(random_model, test_data)
            random_mea.append(abs.mean().item())
        random_result = np.mean(random_mea)

        # model
        train(model, 
            train_data,
            criterion, 
            optimizer,
            config.NUM_ITERS)
        
        abs = test(model, test_data)
        result = abs.mean().item()
        normalized_result = 100.0 * result/random_result

        print('model_type:', config.model)
        print('function:', config.function)
        print('test_result:', result)
        print('normalized_test_result:', normalized_result)

        ###wand plots
        # id = FUNC_id[config.function]

        table_res = wandb.Table(data=[[config.function, result]], 
                                columns=["FUNCTION", "Mean_of_MAE"])
        
        table_norm_res = wandb.Table(data=[[config.function, normalized_result]], 
                                     columns=["FUNCTION", "Scaled_Mean_of_MAE"])

        wandb.log({
        'Results': wandb.plot.bar(table_res, 
                                "FUNCTION",
                                "Mean_of_MAE",
                                title="Results"),
                   
        'Normalized_Results': wandb.plot.bar(table_norm_res, 
                                            "FUNCTION",
                                            "Scaled_Mean_of_MAE", 
                                            title="Normalized_Results")
        })

    return model


def make(config):
    # Make the model
    model = MODELS[config.model](config.NUM_LAYERS, 
                                config.in_dim, 
                                config.HIDDEN_DIM, 
                                config.out_dim).model

    random_model = MLP(config.NUM_LAYERS, 
                        config.in_dim, 
                        config.HIDDEN_DIM, 
                        config.out_dim).model

    # Make the data
    data = generate_data(config.function,
                         config.num_train, 
                         config.num_test, 
                         config.dim, 
                         config.num_sum, 
                         config.RANGE)
    
    # Make the loss and optimizer
    criterion = F.mse_loss
    optimizer = torch.optim.RMSprop(model.parameters(), lr=config.LEARNING_RATE)

    # return random_model, model, criterion, optimizer
    return random_model, model, data, criterion, optimizer


def generate_data(function, num_train, num_test, dim, num_sum, support):

    fn = ARITHMETIC_FUNCTIONS[function]
    data = torch.FloatTensor(dim).uniform_(*support).unsqueeze_(1)
    X, y = [], []
    for i in range(num_train + num_test):
        idx_a = random.sample(range(dim), num_sum)
        idx_b = random.sample([x for x in range(dim) if x not in idx_a], num_sum)
        a, b = data[idx_a].sum(), data[idx_b].sum()
        X.append([a, b])
        y.append(fn(a, b))
    X = torch.FloatTensor(X)
    y = torch.FloatTensor(y).unsqueeze_(1)
    indices = list(range(num_train + num_test))
    np.random.shuffle(indices)
    X_train, y_train = X[indices[num_test:]], y[indices[num_test:]]
    X_test, y_test = X[indices[:num_test]], y[indices[:num_test]]

    return ((X_train, y_train), (X_test, y_test))


def train(model, train_data, criterion, optimizer, num_iters):
    wandb.watch(model, criterion, log="all", log_freq=1000)

    data, target = train_data
    # model.train()
    for i in range(num_iters):
        out = model(data)
        loss = criterion(out, target)
        mea = torch.mean(torch.abs(target - out))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 1000 == 0:
            train_log(loss, mea, i, num_iters)

def train_log(loss, mea, epoch, num_iters):
    wandb.log({"epoch": epoch, "loss_(mse)": loss.item(), 'mean_absolute_error': mea.item()})
    print("\t{}/{}: loss: {:.7f} - mea: {:.7f}".format(epoch+1, num_iters, loss.item(), mea.item()))


def test(model, test_data):
    data, target = test_data
    with torch.no_grad():
        out = model(data)
        return torch.abs(target - out)